In [ ]:
import torch
import torchvision
from torchvision import transforms as T
import torchvision.datasets as dset
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import os
import random
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/new_data_set.zip'

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: content/new_validation_set/0731_c5_020805710.jpg  
  inflating: content/new_validation_set/0474_c3_055032921.jpg  
  inflating: content/new_validation_set/0658_c4_066815997.jpg  
  inflating: content/new_validation_set/0333_c5_070013922.jpg  
  inflating: content/new_validation_set/0333_c6_083620357.jpg  
  inflating: content/new_validation_set/1034_c2_022673018.jpg  
  inflating: content/new_validation_set/0157_c6_012297531.jpg  
  inflating: content/new_validation_set/1277_c3_065333035.jpg  
  inflating: content/new_validation_set/0870_c6_047150495.jpg  
  inflating: content/new_validation_set/0837_c3_072811598.jpg  
  inflating: content/new_validation_set/0271_c1_061621584.jpg  
  inflating: content/new_validation_set/0837_c6_067122953.jpg  
  inflating: content/new_validation_set/1046_c2_012285901.jpg  
  inflating: content/new_validation_set/0113_c1_064691834.jpg  
  inflating: content/new_validation_set/0474_c5

In [ ]:

train_folder = "/content/train/"
train_group_folder = "/content/content/new_train_set_group/"
validation_group_folder = "/content/content/new_validation_set_group/"
check_point_path="/content/drive/MyDrive/Colab Notebooks/check_point/" 
num_epochs = 100 


In [ ]:
def initialize_vgg16(num_classes):
  # load the pre-trained Alexnet
  vgg16 = torchvision.models.vgg16()
  # get the number of neurons in the penultimate layer
  in_features = vgg16.classifier[6].in_features
  
  # re-initalize the output layer
  vgg16.classifier[6] = torch.nn.Linear(in_features=in_features, 
                                          out_features=num_classes)
  
  return vgg16

In [ ]:
def initialize_alexnet(num_classes):
  # load the pre-trained Alexnet
  alexnet = torchvision.models.alexnet(pretrained=True)
  
  # get the number of neurons in the penultimate layer
  in_features = alexnet.classifier[6].in_features
  
  # re-initalize the output layer
  alexnet.classifier[6] = torch.nn.Linear(in_features=in_features, 
                                          out_features=num_classes)
  
  return alexnet

In [ ]:
def initialize_resnet(num_classes):
  # load the pre-trained Alexnet
  resnet =  torchvision.models.resnet50(pretrained=True)
  # get the number of neurons in the penultimate layer
  in_features = resnet.fc.in_features
  
  # re-initalize the output layer
  resnet.fc = torch.nn.Linear(in_features=in_features, 
                                          out_features=num_classes)
  
  return resnet

Loss function is triplet loss 

In [ ]:
def get_cost_function():
  cost_function = torch.nn.TripletMarginLoss(margin=1.0, p=2)
  return cost_function

In [ ]:
def get_optimizer(net, lr, wd, momentum):
  optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
  return optimizer

In [ ]:
test_folder = "/content/content/val_test/"
query_folder = "/content/content/val_queries/"
net_type="vgg16-new-test"
device='cuda:0'
learning_rate=0.01
weight_decay=0.000001
momentum=0.9

In [ ]:
# Prepare data transformations and then combine them sequentially
transform = list()
#transform.append(T.ToTensor())                            # converts Numpy to Pytorch Tensor
#transform.append(T.Normalize(mean=[0.5], std=[0.5]))      # Normalizes the Tensors between [-1, 1]
#transform = T.Compose(transform)                          # Composes the above transformations into one.



transform.append(T.Resize((256, 256)))                      # Resize each PIL image to 256 x 256
transform.append(T.RandomCrop((224, 224)))                  # Randonly crop a 224 x 224 patch
transform.append(T.ToTensor())                              # converts Numpy to Pytorch Tensor
transform.append(T.Normalize(mean=[0.485, 0.456, 0.406], 
                              std=[0.229, 0.224, 0.225]))    # Normalize with ImageNet mean
transform = T.Compose(transform)
checkpoint = torch.load(check_point_path+net_type+"_max"+".pt")
net =initialize_vgg16(512).to(device)
optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['val_loss']
print(epoch)
print(loss)



30
0.015345124436749352


In [ ]:
def caculate_ap(example, example_labels, dict_emb):
  distance_dict={}
  min=1000
  for x in dict_emb.keys():
    value=np.linalg.norm(dict_emb[x]-example)
    distance_dict[x]=value
  list_candidate=dict_emb.keys()
  list_sort_candidate=sorted(list_candidate, key=lambda x: distance_dict[x])
  ap=0
  tp=0
  for idx, filename in enumerate(list_sort_candidate):
    person_id=filename.split("_")[0]
    if person_id==example_labels:
      tp=tp+1
      ap=ap+tp/(idx+1)
  return ap/tp



def validate(net,transform,device):
  net.eval()
  final_test_examples=os.listdir(test_folder)
  final_query_examples=os.listdir(query_folder)
  
  test_emb={}
  with torch.no_grad():  
    for filename in final_test_examples:
      person=filename.split("_")[0]
      img=Image.open(os.path.join(test_folder,filename))
      tensor_img=transform(img).to(device)
      tensor_img.unsqueeze_(0)
      output=net(tensor_img).cpu().numpy()
      test_emb[filename]=output
  total_ap=0
  with torch.no_grad():  
    for filename in final_query_examples:
      print(filename)
      person=filename.split("_")[0]
      img=Image.open(os.path.join(query_folder,filename))
      tensor_img=transform(img).to(device)
      tensor_img.unsqueeze_(0)
      output=net(tensor_img).cpu().numpy()
      person_id=filename.split("_")[0]
      ap=caculate_ap(output,person_id,test_emb)
      total_ap=total_ap+ap
  return total_ap/len(final_query_examples)

    

In [ ]:
print(validate(net,transform,device))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


1282_c6_000787120.jpg
0264_c3_048586173.jpg
1284_c1_003685324.jpg
1244_c1_059108613.jpg
0264_c6_066310719.jpg
0900_c5_038949467.jpg
1095_c1_094475593.jpg
0566_c1_074551917.jpg
1411_c4_083347892.jpg
0347_c6_016233915.jpg
1053_c6_065004690.jpg
0162_c1_023493269.jpg
0141_c5_026338731.jpg
0097_c6_064755042.jpg
1169_c6_073950340.jpg
0900_c3_006145691.jpg
0257_c3_052818276.jpg
1461_c3_038963836.jpg
1466_c6_003473635.jpg
0463_c5_024955446.jpg
0035_c4_076229053.jpg
1052_c3_089496377.jpg
0847_c3_056749683.jpg
1204_c2_032227180.jpg
1232_c6_009581142.jpg
0758_c5_079573973.jpg
0474_c2_064463567.jpg
1055_c6_072432779.jpg
0182_c4_071746754.jpg
1487_c5_077655869.jpg
0488_c5_080313755.jpg
0429_c6_068524840.jpg
1319_c4_072426843.jpg
0283_c3_078571982.jpg
1056_c5_042546270.jpg
0363_c1_093961862.jpg
0644_c3_062475671.jpg
1210_c6_086489816.jpg
1464_c1_042847145.jpg
0422_c5_046662490.jpg
0470_c5_026804488.jpg
0765_c6_018082735.jpg
0331_c2_018218440.jpg
1138_c2_082784385.jpg
0005_c3_012765105.jpg
0130_c6_04